# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [3]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()

In [4]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [5]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [6]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [7]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew, as you mentioned earlier.'

In [8]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is Andrew, as you mentioned earlier.


In [9]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2.\nHuman: What is my name?\nAI: Your name is Andrew, as you mentioned earlier."}

In [10]:
memory = ConversationBufferMemory()

In [11]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [12]:
print(memory.buffer)

Human: Hi
AI: What's up


In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [14]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [15]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [16]:
from langchain.memory import ConversationBufferWindowMemory

In [17]:
memory = ConversationBufferWindowMemory(k=1)

In [18]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [19]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [20]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [21]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [22]:
conversation.predict(input="What is 1+1?")

'The answer to 1+1 is 2.'

In [23]:
conversation.predict(input="What is my name?")

"I'm sorry, I don't have access to that information. Could you please tell me your name?"

In [24]:
print(memory.buffer)

[HumanMessage(content='Hi, my name is Andrew', additional_kwargs={}, example=False), AIMessage(content="Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?", additional_kwargs={}, example=False), HumanMessage(content='What is 1+1?', additional_kwargs={}, example=False), AIMessage(content='The answer to 1+1 is 2.', additional_kwargs={}, example=False), HumanMessage(content='What is my name?', additional_kwargs={}, example=False), AIMessage(content="I'm sorry, I don't have access to that information. Could you please tell me your name?", additional_kwargs={}, example=False)]


## ConversationTokenBufferMemory

In [25]:
# !pip install tiktoken

In [26]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [27]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [28]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [29]:
from langchain.memory import ConversationSummaryBufferMemory

In [30]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [31]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments."}

In [32]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [33]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments.
Human: What would be a good demo to show?
AI:

> Finished chain.


"Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can accurately understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our AI's ability to learn and adapt over time, making it a valuable tool for businesses looking to improve their customer experience."

In [34]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments. The human asks what would be a good demo to show.\nAI: Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can accurately understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our AI's ability to learn and adapt over time, making it a valuable tool for businesses looking to improve their customer experience."}

In [35]:
# create a long string
schedule1 = "오전 8시에 제품 팀과 회의가 있습니다. \
파워포인트 프레젠테이션을 준비해야 합니다. \
오전 9시부터 오후 12시까지 LangChain 프로젝트 작업을 할 시간이 있는데,\
LangChain은 매우 강력한 도구이기 때문에 빠르게 진행될 것입니다. \
정오에는 최신 AI를 이해하기 위해 한 시간 넘게 운전해 온 고객과 함께 이탈리안 레스토랑에서 점심을 먹게 됩니다. \
최신 LLM 데모를 보여드리기 위해 노트북을 지참하는 것을 잊지 마세요."

memory1 = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory1.save_context({"input": "Hello"}, {"output": "What's up"})
memory1.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory1.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule1}"})

In [36]:
memory1.load_memory_variables({})

{'history': "System: The human greets the AI and asks what's on the schedule for the day. The AI responds with a list of tasks, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to understand the latest AI. The AI reminds the human to bring their laptop to show the latest LLM demo."}

In [37]:
conversation1 = ConversationChain(
    llm=llm, 
    memory = memory1,
    verbose=True
)

In [38]:
conversation1.predict(input="어떤 데모를 보여주면 좋을까요?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks what's on the schedule for the day. The AI responds with a list of tasks, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to understand the latest AI. The AI reminds the human to bring their laptop to show the latest LLM demo.
Human: 어떤 데모를 보여주면 좋을까요?
AI:

> Finished chain.


'그것은 당신이 보여줄 수 있는 것에 따라 다릅니다. 그러나 최신 LLM 데모를 보여주는 것이 좋을 것입니다. 그것은 우리의 최신 기술 중 하나이며, 우리의 제품 개발에 매우 중요합니다. 그리고 우리의 고객들은 항상 우리의 최신 기술에 대해 관심이 많으므로 그것을 보여주는 것이 좋습니다.'

In [39]:
memory1.load_memory_variables({})

{'history': "System: The human greets the AI and asks about the day's schedule. The AI responds with a list of tasks, including a meeting with the product team, working on the LangChain project, and having lunch with a customer to understand the latest AI. The AI suggests showing the latest LLM demo, which is important for product development and of interest to customers."}